In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
from dgfip.read import get_structures
structures = get_structures()

# Variables disponibles

In [3]:
structures.columns

Index(['Nomenclature interne', 'TYPE DE SERVICE', 'LIBELLE DU SERVICE',
       'NOM DU SERVICE', 'DEPARTEMENT', 'ADRESSE', 'BP', 'CODE POSTAL',
       'COMMUNE', 'CODE COMMUNE', 'COURRIEL', 'HORAIRES 1', 'ACCUEIL SUR RDV',
       'geocodage', 'Type de démarche', 'TELEPHONE 1', 'TELEPHONE 2',
       'Nom Officiel Département', 'Code Officiel Région',
       'Nom Officiel Région', 'public', 'service', 'Code Courant Officiel',
       'Nom Officiel Commune', 'dep_code_de_geocodage', 'geometry'],
      dtype='object')

# Horaires

In [4]:
structures['HORAIRES 1']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
17834    NaN
17835    NaN
17836    NaN
17837    NaN
17838    NaN
Name: HORAIRES 1, Length: 17292, dtype: object

In [5]:
# fond de carte des départements
dep = gpd.read_file('https://github.com/gregoiredavid/france-geojson/raw/master/departements-version-simplifiee.geojson', crs='4236')

In [6]:
count = structures.groupby(['DEPARTEMENT','TYPE DE SERVICE'], as_index=False).size()
tot = count[count['TYPE DE SERVICE'] != 'Buralistes'].groupby('DEPARTEMENT').sum().reset_index()
tot['TYPE DE SERVICE'] = 'Non Buralistes'
base = pd.concat((count, tot[['DEPARTEMENT', 'TYPE DE SERVICE', 'size']]))
# import de la population française
population = pd.read_csv('../data/ensemble/Departements.csv', sep=";")
count_norm = base.merge(population, left_on='DEPARTEMENT', right_on="CODDEP")
count_norm['par_hab'] = count_norm['size']/count_norm['PTOT']*10000
base = count_norm[['DEPARTEMENT','TYPE DE SERVICE', 'par_hab','size']]
data = dep.merge(count_norm, left_on = 'code', right_on = 'DEPARTEMENT')


In [10]:
# import de la population française
population = pd.read_csv('../data/ensemble/Departements.csv', sep=";")

FileNotFoundError: [Errno 2] No such file or directory: '../data/ensemble/Departements.csv'

In [ ]:
count_norm = base.merge(population, left_on='DEPARTEMENT', right_on="CODDEP")
count_norm['par_hab'] = count_norm['size']/count_norm['PTOT']*10000

In [ ]:
base = count_norm[['DEPARTEMENT','TYPE DE SERVICE', 'par_hab','size']]

In [ ]:
base.to_csv('../data/dep_count.csv')

In [ ]:
data = dep.merge(count_norm, left_on = 'code', right_on = 'DEPARTEMENT')

In [ ]:
base = data[['code', 'nom','geometry', 'TYPE DE SERVICE', 'par_hab']]

In [ ]:
base.to_file('../data/count_departement.geojson',driver="GeoJSON")

/home/titouan/projects/OptimisationFiscale/.venv/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
